# UV-Vis Data Collection Project

In [2]:
import pandas as pd

Below a table will be constructed containing the UV-vis data from a crystal violet experiment. There will be five reactions with varying concentrations of NaOH in M. The slope represents the absorbance over time, as the violet color from crystal violet fades over time as the reaction proceeds.

In [3]:
uvvis = pd.read_csv("crystalvioletformatted", delim_whitespace=False,
                    usecols=['Reaction','hydroxide','Time','Abs'], parse_dates=True)
print(uvvis)

     Reaction  hydroxide        Time      Abs
0         1.0       0.10    0.312425  0.04874
1         1.0       0.10    1.546510  0.04840
2         1.0       0.10    2.796216  0.04744
3         1.0       0.10    4.030302  0.04664
4         1.0       0.10    5.280008  0.04594
..        ...        ...         ...      ...
944       5.0       0.02  355.288854  0.04349
945       5.0       0.02  356.522939  0.04339
946       5.0       0.02  357.757023  0.04329
947       5.0       0.02  358.991108  0.04329
948       5.0       0.02  360.225193  0.04310

[949 rows x 4 columns]


/var/folders/ks/zxyb8jx94md2pr1d256fk17m0000gn/T/ipykernel_75262/944261660.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  uvvis = pd.read_csv("crystalvioletformatted", delim_whitespace=False,


The concentration of crystal violet in Molar was calculated using Beer's Law and added to the table under the column "cv".

In [4]:
uvvis["cv"]=uvvis["Abs"]/87000
print(uvvis)


     Reaction  hydroxide        Time      Abs            cv
0         1.0       0.10    0.312425  0.04874  5.602299e-07
1         1.0       0.10    1.546510  0.04840  5.563218e-07
2         1.0       0.10    2.796216  0.04744  5.452874e-07
3         1.0       0.10    4.030302  0.04664  5.360920e-07
4         1.0       0.10    5.280008  0.04594  5.280460e-07
..        ...        ...         ...      ...           ...
944       5.0       0.02  355.288854  0.04349  4.998851e-07
945       5.0       0.02  356.522939  0.04339  4.987356e-07
946       5.0       0.02  357.757023  0.04329  4.975862e-07
947       5.0       0.02  358.991108  0.04329  4.975862e-07
948       5.0       0.02  360.225193  0.04310  4.954023e-07

[949 rows x 5 columns]


## Curve Fitting 

In [5]:
import numpy as numpy

This imports a collection of functions know as numpy, and I named it as numpy as well to keep track of it.

In [6]:
from scipy.optimize import curve_fit

This imports the curve fit function from the SciPy reference manual. This specific function was found under the scipy.optimize section which includes methods of solving nonlinear data sets, though I am also using it for a linear set for the sake of consistency.

In [7]:
def first(x, a, b):
    return a * numpy.exp(-b * x)

This code defines a function that I have titled "first" and it will be used to fit the data to a first-order, exponential line. The parameter "a" is the initial concentration, "b" is the rate constant k, and "x" is the time. Follows the equation: [A]=[A0]e^(-kt)

In [8]:
def zero(x, a, b):
    return -a * x + b

This code defines a function that I have titled "zero" and it will be used to fit the data to a zero-order, linear line. The parameter "a" is the rate constant k, "b" is the initial concentration, and "x" is the time. Follows the equation: [A]=-kt+[A0]

In [9]:
def second(x, a, b):
    return (a) / (a * b * x + 1)

This code defines a function that I have titled "second" and it will be used to fit the data to a second-order line. The parameter "a" is the initial concentration, "b" is the rate constant k, and "x" is time. Follows the equation: [A]=[A0]/([A0]kt+1)

#### Reaction 1

The following code blocks define the x and y variables for the data points collected in Reaction 1. X is defined as the time values, while y is defined as the crystal violent concentration values.

In [10]:
xdata1 = uvvis.loc[uvvis["Reaction"]==1,"Time"]

In [11]:
ydata1 = uvvis.loc[uvvis["Reaction"]==1,"cv"]

The code block below fits the assigned variables to the zero order function as defined above using the curve fit function. This function's output is the optimized parameters, a and b. "popt10" refers to the optimized parameters for reaction 1 fitted to zero order.

In [12]:
popt10, pcov10 = curve_fit(zero, xdata1, ydata1)
popt10

array([3.20547042e-09, 5.05761593e-07])

The code block below fits the assigned variables to the first order function defined above and the optimized parameters are given. "popt11" refers to the optimized parameters of reaction 1 fitted to first order.

In [13]:
popt11, pcov11 = curve_fit(first, xdata1, ydata1)
popt11

array([5.52821793e-07, 1.06013483e-02])

The code block below fits the assigned variables to the second order function defined above and the optimized parameters are given. "popt12" refers to the optimized parameters of reaction 1 fitted to the second order.

In [14]:
popt12, pcov12 = curve_fit(second, xdata1, ydata1)
popt12

array([5.94409730e-07, 3.09059033e+04])

### Finding the AIC Value

Matplotlib will help visualize all three orders and reactions!

In [24]:
import matplotlib.pyplot as plt

The AIC, or Akaike Information Criterion, is a tool used to test the goodness of fit for different models. A lower score indicates that the fit is better. The goal is to determine which reaction order best fits the data. The following code block helps assign an Akaike value to each order (shout out to Dr. Marsh). 

In [25]:
def getAIC(x, y, model, params):
    #get predictions using the fitted parameters
    y_pred = model(x, *params)

    #residual sum of squares
    rss = np.sum ((y-y_pred) ** 2)

    #number of data points
    n = len (y)

    #of parameters estimated (k)
    k = len(params) # a and b => k = 2

    #log-likelihood under assumption of Gaussian errors
    log_likelihood = -0.5 * n * (np.log(2 * np.pi *rss / n) + 1)

    #AIC formula
    return 2 * k -2 * log_likelihood

The purpose of the following code is to obtain the AIC number and to visualize what the different orders look like verses the actual obtained data.

In [ ]:
params, cov = curve_fit(zero, X, y)
AIC0 = getAIC(X, y, zero, params)
print ("Zero AIC: ", AIC0)
plt.plot(X, zero(X, *params), '.', color='blue'